In [1]:
import os

import matplotlib.pyplot as plt
import pandas as pd

from sklearn.datasets import load_iris

OUTPUT_FOLDER = 'data'

data = load_iris()
X = data.data
y = data.target

# os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# for i, (yi, img) in enumerate(zip(y, data.images)):
#     fname = os.path.join(OUTPUT_FOLDER, f'{i}.png')
#     if not os.path.exists(fname):
#         plt.imsave(fname, img, cmap='binary', vmin=0, vmax=16)

# Example of how to render a single row in X as a radar chart

In [2]:
import chissl

kwargs = dict(
    features=pd.DataFrame().assign(target=data.target),
    component='img',
    prefix=chissl.get_jupyter_url_from_local_path('data/'),
    suffix='.png'
)

self = chissl.ChisslWidget(X, **kwargs)

# create test data for Storybook
# import json
# with open('../../stories/props.json', 'w') as fp:
#     json.dump(self.props, fp, indent=2)

self

ChisslWidget(component='ChisslWidget', props={'parents': [194, 201, 192, 236, 188, 227, 198, 215, 167, 168, 19…

In [3]:
import json

schema = {
  "$schema": "https://vega.github.io/schema/vega/v5.json",
  "description": "A radar chart example, showing multiple dimensions in a radial layout.",
  "width": 400,
  "height": 400,
  "padding": 40,
  "autosize": {"type": "none", "contains": "padding"},

  "signals": [
    {"name": "radius", "update": "width / 2"}
  ],

  "data": [
    {
      "name": "keys",
      "source": "table",
      "transform": [
        {
          "type": "aggregate",
          "groupby": ["key"]
        }
      ]
    }
  ],

  "scales": [
    {
      "name": "radial",
      "type": "linear",
      "range": {"signal": "[0, radius]"},
      "zero": True,
      "nice": False,
      "domain": {"data": "table", "field": "value"},
      "domainMin": 0
    },
    {
      "name": "angular",
      "type": "point",
      "range": {"signal": "[-PI, PI]"},
      "padding": 0.5,
      "domain": {"data": "table", "field": "key"}
    },
    {
      "name": "color",
      "type": "ordinal",
      "domain": ["focus", "ref"],
      "range": ["red", "lightgray"]
    }
  ],

  "encode": {
    "enter": {
      "x": {"signal": "radius"},
      "y": {"signal": "radius"}
    }
  },

  "marks": [
    {
      "type": "group",
      "name": "categories",
      "zindex": 1,
      "from": {
        "facet": {"data": "table", "name": "facet", "groupby": ["category"]}
      },
      "marks": [
        {
          "type": "line",
          "name": "category-line",
          "from": {"data": "facet"},
          "encode": {
            "enter": {
              "interpolate": {"value": "linear-closed"},
              "x": {"signal": "scale('radial', datum.value) * cos(scale('angular', datum.key))"},
              "y": {"signal": "scale('radial', datum.value) * sin(scale('angular', datum.key))"},
              "stroke": {"scale": "color", "field": "category"},
              "strokeWidth": {"value": 1},
              "fill": {"scale": "color", "field": "category"},
              "fillOpacity": {"value": 0.4}
            }
          }
        }
      ]
    },
    {
      "type": "rule",
      "name": "radial-grid",
      "from": {"data": "keys"},
      "zindex": 0,
      "encode": {
        "enter": {
          "x": {"value": 0},
          "y": {"value": 0},
          "x2": {"signal": "radius * cos(scale('angular', datum.key))"},
          "y2": {"signal": "radius * sin(scale('angular', datum.key))"},
          "stroke": {"value": "lightgray"},
          "strokeWidth": {"value": 1}
        }
      }
    },
    {
      "type": "line",
      "name": "outer-line",
      "from": {"data": "radial-grid"},
      "encode": {
        "enter": {
          "interpolate": {"value": "linear-closed"},
          "x": {"field": "x2"},
          "y": {"field": "y2"},
          "stroke": {"value": "lightgray"},
          "strokeWidth": {"value": 1}
        }
      }
    }
  ]
}


def vega_data(focus, reference):
    def make_entry(n,v, label): 
        return {"key": f"key-{n}", "value": v, "category": label}
    
    focus_values = [make_entry(i,v, "focus") for i,v in enumerate(focus)]
    ref_values = [make_entry(i,v, "ref") for i,v in enumerate(reference)]
    return {"name": "table", "values": ref_values + focus_values}

In [4]:
from pathlib import Path

#you can paste files to https://vega.github.io/editor/#/edited

def write_many(X, root):
    os.makedirs(root, exist_ok=True)
    root = Path(root)
    
    abs_max = X.max()
    local_schema = schema["scales"][0]['domain'] = [0, abs_max]
    
    for i,x in enumerate(X):
        one = schema.copy()
        one['data'] = [vega_data(x, X.mean(axis=0))] + schema['data']
        with open(root/f"{i}.json", "w") as f: 
            json.dump(one,f, indent=2)

write_many(X, "./data")

In [5]:
!cat data/0.json

{
  "$schema": "https://vega.github.io/schema/vega/v5.json",
  "description": "A radar chart example, showing multiple dimensions in a radial layout.",
  "width": 400,
  "height": 400,
  "padding": 40,
  "autosize": {
    "type": "none",
    "contains": "padding"
  },
  "signals": [
    {
      "name": "radius",
      "update": "width / 2"
    }
  ],
  "data": [
    {
      "name": "table",
      "values": [
        {
          "key": "key-0",
          "value": 5.843333333333335,
          "category": "ref"
        },
        {
          "key": "key-1",
          "value": 3.057333333333334,
          "category": "ref"
        },
        {
          "key": "key-2",
          "value": 3.7580000000000027,
          "category": "ref"
        },
        {
          "key": "key-3",
          "value": 1.199333333333334,
          "category": "ref"
        },
        {
          "key": "key-0",
          "value": 5.1,
          "category": "focus"
        },
        {
          "key": "key-1"